In [ ]:
from gurobipy import *

number_job = 10  # number_job
number_machine = 3 
processing_time = [3,5,1,3,2,4,5,6,4,4]
# resource_consumption = [4,2,3,3,1,1,2,3,1,1,1,4]
# Job_successors = [[1,2,3],[4],[5,7],[5],[6],[7,8,9],[10],[10],[11],[11],[11],[]]


# Initial the gurobi model
m = Model()

#add variables 
x = m.addVars(number_machine,number_job,vtype=GRB.BINARY, name='x')
Cmax = m.addVar(lb = 0, name='makespan')

# Set the obj which means the minmize the makespan
m.setObjective(Cmax,GRB.MINIMIZE)

# add constraints
for i in range(number_machine):
    m.addConstr(quicksum(x[i,j]*processing_time[j] for j in range(number_job)) <= Cmax) 

for j in range(number_job):
    m.addConstr(quicksum(x[i,j] for i in range(number_machine)) == 1) 

m.optimize()


# Get the solution
output_list=[]
vars =  m.getVars()
for i in range(number_machine*number_job):
#     print('%s %g' % (vars[i].varName, vars[i].x))
    output_list.append(vars[i].x)

    
    
# construct the 2-d array to store the jobs that process on each machine
machine_list =[]
for i in range(number_machine):
    machine_list.append([])
for i in range(len(output_list)):
    if(output_list[i] == 1):
        machine_list[i//number_job].append(i%number_job)

# print (machine_list)

# get the start date/finish date of each job
m_time = [0] *number_machine
start =[0]  * number_job
finish =[0] * number_job
for i in range(len(machine_list)):
    for j in range(len(machine_list[i])):
        start[machine_list[i][j]] = m_time[i]
        finish[machine_list[i][j]] = m_time[i]+processing_time[machine_list[i][j]]
        m_time[i] = finish[machine_list[i][j]]

# Print the results （absolute time）
print (start)
print (finish)

In [ ]:
from datetime import datetime  
from datetime import timedelta

import plotly.express as px
import pandas as pd

# organize the data (absolute date time) to plotly
# suppose we use 2020-01-01 as the baseline time
start_date=[]
finish_date =[]

start_date_base = datetime.strptime('2020-01-01','%Y-%m-%d')

for i in range(number_job):
    start_date.append((start_date_base + timedelta(days=start[i])).strftime('%Y-%m-%d'))
    finish_date.append((start_date_base + timedelta(days=finish[i])).strftime('%Y-%m-%d'))

row_list = []
Job_index =0
for i in range(len(machine_list)):
    for j in range(len(machine_list[i])):
        dict1 = dict(Machine = "Machine"+str(i+1), Job  = "Job"+str(machine_list[i][j]+1), Start = start_date[machine_list[i][j]], Finish = finish_date[machine_list[i][j]])
        row_list.append(dict1)

df = pd.DataFrame(row_list)
print (df)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color = 'Job')
fig.update_layout(showlegend=False) 
fig.show()